In [12]:
import pandas as pd

# 한글 폰트 깨짐 방지
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

#에러 무시
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

import matplotlib.pyplot as plt
import matplotlib

from sklearn.tree import export_graphviz
import graphviz

from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import scale, minmax_scale, robust_scale

import xgboost as xgb

import seaborn as sns

plt.rcParams['font.family'] = 'NanumGothic'


In [2]:
hts = pd.read_csv('hts.csv', encoding = 'CP949')
hitter = pd.read_csv('타자 기본.csv', encoding = 'CP949')
player = pd.read_csv('선수.csv', encoding = 'CP949')

In [3]:
sample = hts.loc[(hts['타격결과'] == '홈런') & (hts['연도'] == 2018) & (hts['타자 팀 코드'] == 'HH')]

In [4]:
sample.reset_index(drop = True, inplace = True)

In [5]:
sample

,연도,경기코드,타구코드,선수코드,타자 팀 코드,이닝,타구속도,발사각도,타격결과,투구구속,해당구장
0,2018,20180325HHWO0,180325_163050,62700,HH,8,153.87,35.1,홈런,143.25,고척
1,2018,20180327HHNC0,180327_213109,74729,HH,8,157.69,30.0,홈런,135.48,마산
2,2018,20180327HHNC0,180327_213301,68730,HH,8,153.95,23.6,홈런,143.28,마산
3,2018,20180328HHNC0,180328_191942,76753,HH,3,148.10,26.8,홈런,140.96,마산
4,2018,20180330SKHH0,180330_204344,71752,HH,6,152.82,34.5,홈런,119.11,대전
...,...,...,...,...,...,...,...,...,...,...,...
133,2018,20181009HHKT0,181009_142927,79705,HH,2,140.41,36.7,홈런,123.25,수원
134,2018,20181009HHKT0,181009_151103,74729,HH,3,158.80,27.6,홈런,136.50,수원
135,2018,20181009HHKT0,181009_164055,73136,HH,7,154.37,28.4,홈런,120.83,수원
136,2018,20181010HHHT0,181010_212916,73136,HH,9,156.31,29.2,홈런,145.97,광주


In [6]:
sample['비거리'] = 0

In [7]:
sample.to_csv('홈런 비거리 샘플.csv', encoding = 'CP949')

In [8]:
hr = pd.read_csv('홈런 비거리 샘플.csv', encoding = 'CP949')

In [9]:
hr.drop('Unnamed: 0', axis = 1, inplace = True)

In [10]:
hr_sample = hr[['타구속도', '발사각도', '비거리']]

## 비거리 회귀 모델

In [28]:
df = pd.read_csv('홈런 비거리 샘플 train.csv', encoding = 'CP949')

In [29]:
df_raw_y = df[['비거리']]# 목표변수
df_raw_x = df[['타구속도','발사각도']] # 독립변수

In [30]:
# train, test set 나누기
df_train_x, df_test_x, df_train_y, df_test_y = train_test_split(df_raw_x, df_raw_y, test_size = 0.4, random_state = 1234)

print('train data x size : {}'.format(df_train_x.shape))
print('train data y size : {}'.format(df_train_y.shape))
print('test data x size : {}'.format(df_test_x.shape))
print('test data y size : {}'.format(df_test_y.shape))

train data x size : (42, 2)
train data y size : (42, 1)
test data x size : (29, 2)
test data y size : (29, 1)


In [31]:
# 기본 성능 모델
gb_uncustomized = xgboost.XGBRegressor(random_state = 1234)
gb_uncustomized.fit(df_train_x, df_train_y)

print('Score on trainning set : {:3f}'.format(gb_uncustomized.score(df_train_x, df_train_y)))
print('Score on test set : {:3f}'.format(gb_uncustomized.score(df_test_x, df_test_y)))

Score on trainning set : 1.000000
Score on test set : 0.052176


In [32]:
# 최종 모델
# gblinear : 선형모형
# gbtree : 의사결정기반모형
# dart : 드롭아웃 regression trees

xgb_final = xgboost.XGBRegressor(random_state = 1234, n_estimators = 300,
                                  colsample_bylevel = 0.5, colsample_bytree = 1, max_depth = 4)

xgb_final.fit(df_train_x, df_train_y)

print('Score on training set: {:.3f}'.format(xgb_final.score(df_train_x, df_train_y)))
print('Score on test set: {:.3f}'.format(xgb_final.score(df_test_x, df_test_y)))

Score on training set: 1.000
Score on test set: 0.014


In [33]:
df_t = pd.read_csv('홈런 비거리 샘플 test.csv', encoding = 'CP949')

In [36]:
df_t = df_t[['타구속도','발사각도']]

In [37]:
pred = xgb_final.predict(df_t)

In [38]:
pred

array([126.525986, 125.598305, 112.32326 , 110.252365, 121.88171 ,
       120.49664 , 119.18214 , 119.81369 , 121.2615  , 119.00562 ,
       125.8964  , 110.0162  , 111.948616, 116.7032  , 118.78052 ,
       121.99985 , 118.50397 , 111.66295 , 106.86294 , 115.05141 ,
       123.91109 , 128.30568 , 111.06512 , 111.66898 , 112.14194 ,
       126.22058 , 119.99824 , 114.52554 , 122.1182  , 112.08409 ,
       120.10959 , 110.14281 , 106.598915, 112.24448 , 103.784004,
       114.50962 , 112.13931 , 114.71588 , 112.19064 , 107.668365,
       114.42525 , 117.47941 , 118.83701 , 122.954384, 121.676674,
       108.50402 , 110.18146 , 123.273575, 113.66659 , 117.3479  ,
       123.265236, 108.53453 , 110.69067 , 110.43922 , 113.919815,
       110.04109 , 120.00171 , 106.29647 , 110.05819 , 125.77205 ,
       125.00262 , 111.56815 , 111.99724 , 123.356064, 125.65861 ,
       121.77374 , 111.7334  , 112.15952 , 120.00261 , 124.99943 ],
      dtype=float32)

In [39]:
df_t['비거리'] = pred

In [40]:
df_t

,타구속도,발사각도,비거리
0,155.08,31.1,126.525986
1,159.14,25.4,125.598305
2,153.59,39.8,112.323257
3,140.16,35.6,110.252365
4,154.99,27.5,121.881706
...,...,...,...
65,156.31,29.2,121.773743
66,151.89,29.9,111.733398
67,148.63,29.8,112.159523
68,158.22,30.2,120.002609


In [43]:
df_t.to_csv('홈런 비거리 샘플 pred.csv', encoding = 'CP949')

- 샘플이 적음